In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os

In [2]:
from datetime import date

# cannot use strftime()'s %B format since it depends on the locale
MONTHS = ["January", "February", "March", "April", "May", "June",
          "July", "August", "September", "October", "November", "December"]

def random_dates(n_dates):
    min_date = date(1000, 1, 1).toordinal()
    max_date = date(9999, 12, 31).toordinal()

    ordinals = np.random.randint(max_date - min_date, size=n_dates) + min_date
    dates = [date.fromordinal(ordinal) for ordinal in ordinals]

    x = [MONTHS[dt.month - 1] + " " + dt.strftime("%d, %Y") for dt in dates]
    y = [dt.isoformat() for dt in dates]
    return x, y

In [3]:
np.random.seed(42)

n_dates = 3
x_example, y_example = random_dates(n_dates)
print("{:25s}{:25s}".format("Input", "Target"))
print("-" * 50)
for idx in range(n_dates):
    print("{:25s}{:25s}".format(x_example[idx], y_example[idx]))

Input                    Target                   
--------------------------------------------------
September 20, 7075       7075-09-20               
May 15, 8579             8579-05-15               
January 11, 7103         7103-01-11               


**Let's get the list of all possible characters in the inputs:**

In [4]:
INPUT_CHARS = "".join(sorted(set("".join(MONTHS) + "0123456789, ")))
INPUT_CHARS

' ,0123456789ADFJMNOSabceghilmnoprstuvy'

**And here's the list of possible characters in the outputs:**

In [5]:
OUTPUT_CHARS = "0123456789-"

In [6]:
def date_str_to_ids(date_str, chars=INPUT_CHARS):
    return [chars.index(c) for c in date_str]

 **Writing a function to convert a string to a list of character IDs**

In [7]:
date_str_to_ids(x_example[0], INPUT_CHARS)

[19, 23, 31, 34, 23, 28, 21, 23, 32, 0, 4, 2, 1, 0, 9, 2, 9, 7]

In [8]:
date_str_to_ids(y_example[0], OUTPUT_CHARS)

[7, 0, 7, 5, 10, 0, 9, 10, 2, 0]

In [9]:
def prepare_date_strs(date_strs, chars=INPUT_CHARS):
    X_ids = [date_str_to_ids(dt, chars) for dt in date_strs]
    X = tf.ragged.constant(X_ids, ragged_rank=1)
    return (X + 1).to_tensor() # using 0 as the padding token ID

def create_dataset(n_dates):
    x, y = random_dates(n_dates)
    return prepare_date_strs(x, INPUT_CHARS), prepare_date_strs(y, OUTPUT_CHARS)

In [10]:
np.random.seed(42)

X_train, Y_train = create_dataset(10000)
X_valid, Y_valid = create_dataset(2000)
X_test, Y_test = create_dataset(2000)

In [11]:
Y_train[0]

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([ 8,  1,  8,  6, 11,  1, 10, 11,  3,  1])>

**First version: a very basic seq2seq model
Let's first try the simplest possible model: we feed in the input sequence, which first goes through the encoder (an embedding layer followed by a single LSTM layer), which outputs a vector, then it goes through a decoder (a single LSTM layer, followed by a dense output layer), which outputs a sequence of vectors, each representing the estimated probabilities for all possible output character.Since the decoder expects a sequence as input, we repeat the vector (which is output by the decoder) as many times as the longest possible output sequence.**

In [13]:
embedding_size = 32
max_output_length = Y_train.shape[1]

np.random.seed(42)
tf.random.set_seed(42)

encoder = keras.models.Sequential([
    keras.layers.Embedding(input_dim=len(INPUT_CHARS) + 1,
                           output_dim=embedding_size,
                           input_shape=[None]),
    keras.layers.LSTM(128)
])

decoder = keras.models.Sequential([
    keras.layers.LSTM(128, return_sequences=True),
    keras.layers.Dense(len(OUTPUT_CHARS) + 1, activation="softmax")
])

model = keras.models.Sequential([
    encoder,
    keras.layers.RepeatVector(max_output_length),
    decoder
])

optimizer = keras.optimizers.Nadam()
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(X_train, Y_train, epochs=20,
                    validation_data=(X_valid, Y_valid))

Epoch 1/20
313/313 [==============================] - 13s 25ms/step - loss: 1.8128 - accuracy: 0.3497 - val_loss: 1.3708 - val_accuracy: 0.4943
Epoch 2/20
313/313 [==============================] - 6s 19ms/step - loss: 1.2090 - accuracy: 0.5590 - val_loss: 1.0403 - val_accuracy: 0.6230
Epoch 3/20
313/313 [==============================] - 7s 21ms/step - loss: 0.9837 - accuracy: 0.6488 - val_loss: 0.8473 - val_accuracy: 0.6943
Epoch 4/20
313/313 [==============================] - 7s 21ms/step - loss: 0.9595 - accuracy: 0.6654 - val_loss: 0.7428 - val_accuracy: 0.7227
Epoch 5/20
313/313 [==============================] - 7s 21ms/step - loss: 0.6312 - accuracy: 0.7608 - val_loss: 0.5384 - val_accuracy: 0.7881
Epoch 6/20
313/313 [==============================] - 7s 21ms/step - loss: 0.4527 - accuracy: 0.8222 - val_loss: 0.3747 - val_accuracy: 0.8536
Epoch 7/20
313/313 [==============================] - 6s 20ms/step - loss: 0.6003 - accuracy: 0.7845 - val_loss: 0.3259 - val_accuracy: 0.873

In [14]:
def ids_to_date_strs(ids, chars=OUTPUT_CHARS):
    return ["".join([("?" + chars)[index] for index in sequence])
            for sequence in ids]

In [25]:
X_new = prepare_date_strs(["November 10, 1970", "July 14, 1789"])

In [26]:
#ids = model.predict_classes(X_new)
ids = np.argmax(model.predict(X_new), axis=-1)
for date_str in ids_to_date_strs(ids):
    print(date_str)

1970-11-10
1789-07-14



**However, since the model was only trained on input strings of length 18 (which is the length of the longest date), it does not perform well if we try to use it to make predictions on shorter sequences.We need to ensure that we always pass sequences of the same length as during training, using padding if necessary. Let's write a little helper function for that**

In [28]:
max_input_length = X_train.shape[1]

def prepare_date_strs_padded(date_strs):
    X = prepare_date_strs(date_strs)
    if X.shape[1] < max_input_length:
        X = tf.pad(X, [[0, 0], [0, max_input_length - X.shape[1]]])
    return X

def convert_date_strs(date_strs):
    X = prepare_date_strs_padded(date_strs)
    #ids = model.predict_classes(X)
    ids = np.argmax(model.predict(X), axis=-1)
    return ids_to_date_strs(ids)

In [30]:
convert_date_strs(["March 01, 1970", "July 14, 1789"])

['1970-03-01', '1789-07-14']

<h1>Feeding the shifted targets to the decoder (teacher forcing)</h1>

**Instead of feeding the decoder a simple repetition of the encoder's output vector, we can feed it the target sequence, shifted by one time step to the right. This way, at each time step the decoder will know what the previous target character was. This should help is tackle more complex sequence-to-sequence problems.**

**Since the first output character of each target sequence has no previous character, we will need a new token to represent the start-of-sequence (sos).**


**let's create the decoder's inputs (for training, validation and testing). The sos token will be represented using the last possible output character's ID + 1.**

In [31]:
sos_id = len(OUTPUT_CHARS) + 1

def shifted_output_sequences(Y):
    sos_tokens = tf.fill(dims=(len(Y), 1), value=sos_id)
    return tf.concat([sos_tokens, Y[:, :-1]], axis=1)

X_train_decoder = shifted_output_sequences(Y_train)
X_valid_decoder = shifted_output_sequences(Y_valid)
X_test_decoder = shifted_output_sequences(Y_test)

In [32]:
X_train_decoder

<tf.Tensor: shape=(10000, 10), dtype=int32, numpy=
array([[12,  8,  1, ..., 10, 11,  3],
       [12,  9,  6, ...,  6, 11,  2],
       [12,  8,  2, ...,  2, 11,  2],
       ...,
       [12, 10,  8, ...,  2, 11,  4],
       [12,  2,  2, ...,  3, 11,  3],
       [12,  8,  9, ...,  8, 11,  3]])>

<h1> Using the functional API </h1>

In [33]:
encoder_embedding_size = 32
decoder_embedding_size = 32
lstm_units = 128

np.random.seed(42)
tf.random.set_seed(42)

encoder_input = keras.layers.Input(shape=[None], dtype=tf.int32)
encoder_embedding = keras.layers.Embedding(
    input_dim=len(INPUT_CHARS) + 1,
    output_dim=encoder_embedding_size)(encoder_input)
_, encoder_state_h, encoder_state_c = keras.layers.LSTM(
    lstm_units, return_state=True)(encoder_embedding)
encoder_state = [encoder_state_h, encoder_state_c]

decoder_input = keras.layers.Input(shape=[None], dtype=tf.int32)
decoder_embedding = keras.layers.Embedding(
    input_dim=len(OUTPUT_CHARS) + 2,
    output_dim=decoder_embedding_size)(decoder_input)
decoder_lstm_output = keras.layers.LSTM(lstm_units, return_sequences=True)(
    decoder_embedding, initial_state=encoder_state)
decoder_output = keras.layers.Dense(len(OUTPUT_CHARS) + 1,
                                    activation="softmax")(decoder_lstm_output)

model = keras.models.Model(inputs=[encoder_input, decoder_input],
                           outputs=[decoder_output])

optimizer = keras.optimizers.Nadam()
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit([X_train, X_train_decoder], Y_train, epochs=10,
                    validation_data=([X_valid, X_valid_decoder], Y_valid))

Epoch 1/10
313/313 [==============================] - 13s 24ms/step - loss: 1.6823 - accuracy: 0.3734 - val_loss: 1.4049 - val_accuracy: 0.4676
Epoch 2/10
313/313 [==============================] - 6s 19ms/step - loss: 1.1923 - accuracy: 0.5552 - val_loss: 0.9020 - val_accuracy: 0.6647
Epoch 3/10
313/313 [==============================] - 7s 21ms/step - loss: 0.6805 - accuracy: 0.7555 - val_loss: 0.4045 - val_accuracy: 0.8665
Epoch 4/10
313/313 [==============================] - 7s 22ms/step - loss: 0.2329 - accuracy: 0.9396 - val_loss: 0.1511 - val_accuracy: 0.9654
Epoch 5/10
313/313 [==============================] - 6s 21ms/step - loss: 0.0772 - accuracy: 0.9907 - val_loss: 0.0428 - val_accuracy: 0.9987
Epoch 6/10
313/313 [==============================] - 6s 18ms/step - loss: 0.0528 - accuracy: 0.9933 - val_loss: 0.0248 - val_accuracy: 0.9991
Epoch 7/10
313/313 [==============================] - 6s 19ms/step - loss: 0.0170 - accuracy: 0.9998 - val_loss: 0.0143 - val_accuracy: 0.999

<h1>This time we need to predict characters one by one</h1>

In [34]:
sos_id = len(OUTPUT_CHARS) + 1

def predict_date_strs(date_strs):
    X = prepare_date_strs_padded(date_strs)
    Y_pred = tf.fill(dims=(len(X), 1), value=sos_id)
    for index in range(max_output_length):
        pad_size = max_output_length - Y_pred.shape[1]
        X_decoder = tf.pad(Y_pred, [[0, 0], [0, pad_size]])
        Y_probas_next = model.predict([X, X_decoder])[:, index:index+1]
        Y_pred_next = tf.argmax(Y_probas_next, axis=-1, output_type=tf.int32)
        Y_pred = tf.concat([Y_pred, Y_pred_next], axis=1)
    return ids_to_date_strs(Y_pred[:, 1:])

In [35]:
predict_date_strs(["July 14, 1789", "May 01, 2020"])

['1789-07-14', '2020-05-01']

<h1>Using TF-Addons's seq2seq implementation</h1>

In [39]:
import tensorflow_addons as tfa

np.random.seed(42)
tf.random.set_seed(42)

encoder_embedding_size = 32
decoder_embedding_size = 32
units = 128

encoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
decoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
sequence_lengths = keras.layers.Input(shape=[], dtype=np.int32)

encoder_embeddings = keras.layers.Embedding(
    len(INPUT_CHARS) + 1, encoder_embedding_size)(encoder_inputs)

decoder_embedding_layer = keras.layers.Embedding(
    len(OUTPUT_CHARS) + 2, decoder_embedding_size)
decoder_embeddings = decoder_embedding_layer(decoder_inputs)

encoder = keras.layers.LSTM(units, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_embeddings)
encoder_state = [state_h, state_c]

sampler = tfa.seq2seq.sampler.TrainingSampler()

decoder_cell = keras.layers.LSTMCell(units)
output_layer = keras.layers.Dense(len(OUTPUT_CHARS) + 1)

decoder = tfa.seq2seq.basic_decoder.BasicDecoder(decoder_cell,
                                                 sampler,
                                                 output_layer=output_layer)
final_outputs, final_state, final_sequence_lengths = decoder(
    decoder_embeddings,
    initial_state=encoder_state)
Y_proba = keras.layers.Activation("softmax")(final_outputs.rnn_output)

model = keras.models.Model(inputs=[encoder_inputs, decoder_inputs],
                           outputs=[Y_proba])
optimizer = keras.optimizers.Nadam()
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit([X_train, X_train_decoder], Y_train, epochs=15,
                    validation_data=([X_valid, X_valid_decoder], Y_valid))

Epoch 1/15
313/313 [==============================] - 12s 23ms/step - loss: 1.6782 - accuracy: 0.3654 - val_loss: 1.4654 - val_accuracy: 0.4414
Epoch 2/15
313/313 [==============================] - 7s 21ms/step - loss: 1.3855 - accuracy: 0.4594 - val_loss: 1.2149 - val_accuracy: 0.5408
Epoch 3/15
313/313 [==============================] - 6s 20ms/step - loss: 0.9870 - accuracy: 0.6298 - val_loss: 0.8000 - val_accuracy: 0.7028
Epoch 4/15
313/313 [==============================] - 6s 19ms/step - loss: 0.4387 - accuracy: 0.8555 - val_loss: 0.2132 - val_accuracy: 0.9531
Epoch 5/15
313/313 [==============================] - 6s 20ms/step - loss: 0.1340 - accuracy: 0.9778 - val_loss: 0.0667 - val_accuracy: 0.9955
Epoch 6/15
313/313 [==============================] - 7s 21ms/step - loss: 0.0941 - accuracy: 0.9857 - val_loss: 0.0469 - val_accuracy: 0.9973
Epoch 7/15
313/313 [==============================] - 6s 19ms/step - loss: 0.0288 - accuracy: 0.9993 - val_loss: 0.0210 - val_accuracy: 0.999

In [40]:
predict_date_strs(["March 01, 1970", "June 18, 2021"])

['1970-03-01', '2021-06-18']